## Objectives

- Establish meaningful grouping of address by clustering DEX traders and LP
    - Feature Extraction:
        - with or without balances at swap time (web3py fetch balance at block)
    - Dimensionality reduction:
        - UMAP / tSNE or PCA / ICA
    - Clustering:
        - DBSCAN
        - silhouette evaluation
    **Deliverable**
    - Visualization:
        - scatter plot with color-coded returns? (TBD)

**Secondary objective: identify which group has the most profitable activity**
- Triggered Average of price by swap in/out by group of addresses

- Predict the return of the next candle based on the "history" of activity of the groups of addresses
    - Feature extracion
        - 

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from py0xcluster.utils.query_utils import *
from py0xcluster.main_classes.pools import *

### Gathering data about most-active pools

In [3]:
uni3pools_selector = PoolSelector(
    subgraph_url = 'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum',
    min_daily_volume_USD = 300000, 
    start_date = (2022,12,20), 
    end_date = (2023,1,9),
    days_batch_size = 5,
    min_days_active = 2)

uni3_pools = uni3pools_selector.get_pools_data(verbose=True)
uni3_pools

Queriying from 2022-12-20 00:00:00 to 2022-12-25 00:00:00
Queriying from 2022-12-25 00:00:00 to 2022-12-30 00:00:00
Queriying from 2022-12-30 00:00:00 to 2023-01-04 00:00:00
Queriying from 2023-01-04 00:00:00 to 2023-01-09 00:00:00
949 lquidity pools snapshots retrieved


,timestamp,dailyVolumeUSD,pool.symbol,pool.totalValueLockedUSD,pool.protocol.name,pool.protocol.network,pool.name,pool.id,token0.symbol,token0.lastPriceUSD,token1.symbol,token1.lastPriceUSD
0,2022-12-20 23:59:23,2.410699e+08,USD Coin/Wrapped Ether,1.796915e+08,Uniswap V3,MAINNET,Uniswap V3 USD Coin/Wrapped Ether 0.05%,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,1.000000,WETH,1340.740814
1,2022-12-19 23:59:59,2.202961e+08,USD Coin/Wrapped Ether,1.796915e+08,Uniswap V3,MAINNET,Uniswap V3 USD Coin/Wrapped Ether 0.05%,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,1.000000,WETH,1340.740814
2,2022-12-23 23:59:59,1.872696e+08,USD Coin/Wrapped Ether,1.796915e+08,Uniswap V3,MAINNET,Uniswap V3 USD Coin/Wrapped Ether 0.05%,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,1.000000,WETH,1340.740814
3,2022-12-22 23:59:47,1.617068e+08,USD Coin/Wrapped Ether,1.796915e+08,Uniswap V3,MAINNET,Uniswap V3 USD Coin/Wrapped Ether 0.05%,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,1.000000,WETH,1340.740814
4,2022-12-21 23:59:47,1.390431e+08,USD Coin/Wrapped Ether,1.796915e+08,Uniswap V3,MAINNET,Uniswap V3 USD Coin/Wrapped Ether 0.05%,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,1.000000,WETH,1340.740814
...,...,...,...,...,...,...,...,...,...,...,...,...
944,2023-01-04 19:17:35,3.107359e+05,Wrapped Ether/Rocket Pool Protocol,2.689047e+06,Uniswap V3,MAINNET,Uniswap V3 Wrapped Ether/Rocket Pool Protocol 1%,0xcf15ad9be9d33384b74b94d63d06b4a9bd82f640,WETH,1340.740814,RPL,25.007020
945,2023-01-03 23:36:35,3.082965e+05,Dai Stablecoin/Wrapped Ether,1.869609e+07,Uniswap V3,MAINNET,Uniswap V3 Dai Stablecoin/Wrapped Ether 0.3%,0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8,DAI,1.000000,WETH,1340.740814
946,2023-01-05 23:58:59,3.065806e+05,WOOL/Wrapped Ether,8.451407e+04,Uniswap V3,MAINNET,Uniswap V3 WOOL/Wrapped Ether 1%,0x7b12d855445073987d45ea97b1af3554f05e4ef4,WOOL,0.000000,WETH,1340.740814
947,2023-01-05 23:31:59,3.063012e+05,Wrapped liquid staked Ether 2.0/Wrapped Ether,1.955933e+06,Uniswap V3,MAINNET,Uniswap V3 Wrapped liquid staked Ether 2.0/Wra...,0xd340b57aacdd10f96fc1cf10e15921936f41e29c,wstETH,1469.290893,WETH,1340.740814


### Filtering pools
    - more exactly filtering snapshots before processing

In [4]:
min_TVL = 100000


uni3_pools = uni3pools_selector.remove_illiquid_pools(uni3_pools, min_TVL)
uni3_pools = uni3pools_selector.remove_stable_pools(uni3_pools)
uni3_pools.head(50)


72 illiquid pools snapshots (over 949) have been removed 
149 stable pools snapshots (over 877) have been removed


,timestamp,dailyVolumeUSD,pool.symbol,pool.totalValueLockedUSD,pool.protocol.name,pool.protocol.network,pool.name,pool.id,token0.symbol,token0.lastPriceUSD,token1.symbol,token1.lastPriceUSD
0,2022-12-20 23:59:23,2.410699e+08,USD Coin/Wrapped Ether,1.796915e+08,Uniswap V3,MAINNET,Uniswap V3 USD Coin/Wrapped Ether 0.05%,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,1.000000,WETH,1340.740814
1,2022-12-19 23:59:59,2.202961e+08,USD Coin/Wrapped Ether,1.796915e+08,Uniswap V3,MAINNET,Uniswap V3 USD Coin/Wrapped Ether 0.05%,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,1.000000,WETH,1340.740814
2,2022-12-23 23:59:59,1.872696e+08,USD Coin/Wrapped Ether,1.796915e+08,Uniswap V3,MAINNET,Uniswap V3 USD Coin/Wrapped Ether 0.05%,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,1.000000,WETH,1340.740814
3,2022-12-22 23:59:47,1.617068e+08,USD Coin/Wrapped Ether,1.796915e+08,Uniswap V3,MAINNET,Uniswap V3 USD Coin/Wrapped Ether 0.05%,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,1.000000,WETH,1340.740814
4,2022-12-21 23:59:47,1.390431e+08,USD Coin/Wrapped Ether,1.796915e+08,Uniswap V3,MAINNET,Uniswap V3 USD Coin/Wrapped Ether 0.05%,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,1.000000,WETH,1340.740814
8,2022-12-20 23:55:47,8.953128e+07,Wrapped BTC/Wrapped Ether,1.098028e+08,Uniswap V3,MAINNET,Uniswap V3 Wrapped BTC/Wrapped Ether 0.05%,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,WBTC,17424.701852,WETH,1340.740814
11,2022-12-22 23:58:35,5.127058e+07,Wrapped Ether/Tether USD,3.467441e+07,Uniswap V3,MAINNET,Uniswap V3 Wrapped Ether/Tether USD 0.05%,0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH,1340.740814,USDT,1.000000
12,2022-12-23 23:59:47,5.102369e+07,Wrapped Ether/Tether USD,3.467441e+07,Uniswap V3,MAINNET,Uniswap V3 Wrapped Ether/Tether USD 0.05%,0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH,1340.740814,USDT,1.000000
13,2022-12-20 23:58:47,4.831932e+07,Wrapped Ether/Tether USD,3.467441e+07,Uniswap V3,MAINNET,Uniswap V3 Wrapped Ether/Tether USD 0.05%,0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH,1340.740814,USDT,1.000000
14,2022-12-19 23:59:59,4.437922e+07,Wrapped Ether/Tether USD,3.467441e+07,Uniswap V3,MAINNET,Uniswap V3 Wrapped Ether/Tether USD 0.05%,0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH,1340.740814,USDT,1.000000


In [339]:
uni3_pools.groupby('pool.symbol').agg('median').sort_values('dailyVolumeUSD', ascending=False).tail(50)

/tmp/ipykernel_58247/1162912073.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  uni3_pools.groupby('pool.symbol').agg('median').sort_values('dailyVolumeUSD', ascending=False).tail(50)


,dailyVolumeUSD,pool.totalValueLockedUSD,token0.lastPriceUSD,token1.lastPriceUSD
pool.symbol,,,,
Offshift/Wrapped Ether,409212.489140,7.872712e+05,1.248826e+00,1313.800574
Numeraire/Wrapped Ether,406380.036708,2.031793e+05,0.000000e+00,1313.800574
Ribbon/USD Coin,405762.139939,8.966205e+04,0.000000e+00,1.000000
Binance USD/Tether USD,405611.698518,4.117402e+05,9.999400e-01,1.000000
South African Tether/Wrapped Ether,405174.171473,1.616449e-13,0.000000e+00,1313.807266
Wrapped Ether/Truebit,404521.808767,1.547862e+05,1.313807e+03,0.000000
Wrapped Ether/Wrapped SOL (Wormhole),404076.665044,8.931331e+04,1.313807e+03,0.000000
Convex Token/USD Coin,403574.814045,1.140000e-04,0.000000e+00,1.000000
UniDex/Wrapped Ether,403542.327344,2.535053e+05,0.000000e+00,1313.800574


In [332]:
uni3_pools.groupby('pool.symbol').agg('median').sort_values('dailyVolumeUSD', ascending=False).head(50)

/tmp/ipykernel_58247/2384805637.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  uni3_pools.groupby('pool.symbol').agg('median').sort_values('dailyVolumeUSD', ascending=False).head(50)


,dailyVolumeUSD,pool.totalValueLockedUSD,token0.lastPriceUSD,token1.lastPriceUSD
pool.symbol,,,,
USD Coin/Wrapped Ether,4.247551e+07,1.350663e+08,1.000000,1296.537802
USD Coin/Tether USD,3.287978e+07,1.194780e+08,1.000000,1.000000
Wrapped BTC/Wrapped Ether,2.804087e+07,1.089433e+08,17224.761416,1296.537802
Wrapped Ether/Tether USD,2.175894e+07,3.462967e+07,1296.537802,1.000000
Dai Stablecoin/USD Coin,6.518340e+06,2.352720e+08,1.000000,1.000000
Dai Stablecoin/Wrapped Ether,6.353363e+06,6.181654e+06,1.000000,1296.537802
Wrapped BTC/USD Coin,5.853777e+06,2.224176e+07,17224.761416,1.000000
ChainLink Token/Wrapped Ether,5.086765e+06,1.927886e+07,6.092683,1296.537802
Rocket Pool/Wrapped Ether,4.813042e+06,2.114466e+05,0.000000,1296.537802
